In [2]:
from geopy.distance import geodesic
import random
import pandas as pd

def generate_parking_locations(center_lat, center_lon, city_radius, downtown_radius, suburb_radius, num_parking_lots, electricity_prices_file):
    parking_lots = []

    # Calculate the number of parking lots in each zone
    # Assume that the number of parking lots in each area is roughly equal
    num_downtown_parking_lots = round(num_parking_lots / 3)
    num_suburb_parking_lots = round(num_parking_lots / 3)
    num_city_center_parking_lots = num_parking_lots - num_downtown_parking_lots - num_suburb_parking_lots

    # Define the probability of business area, office district, Residential district
    downtown_district_probs = {"Business district": 0.4, "Office district": 0.3, "Residential district": 0.3}
    city_district_probs = {"Business district": 0.3, "Office district": 0.4, "Residential district": 0.3}
    suburb_district_probs = {"Business district": 0.2, "Office district": 0.2, "Residential district": 0.6}

    # Read the price of electricity for the day
    df_prices = pd.read_csv(electricity_prices_file, header=None, names=["DateTime", "ElectricityPrice"])
    one_day_prices_list = df_prices["ElectricityPrice"][:24].tolist()

    for _ in range(num_city_center_parking_lots):
        location_type = "city_center"
        # Generates latitude and longitude within the city center
        angle = random.uniform(0, 2 * 3.14159)
        distance = random.uniform(0, downtown_radius)
        destination = geodesic(kilometers=distance).destination((center_lat, center_lon), angle)
        
        # Randomly select the attributes of business district, office district and Residential district
        district_type = random.choices(list(downtown_district_probs.keys()), weights=list(downtown_district_probs.values()))[0]
        
        new_one_day_prices_list = []
        # Set parking fees and the number of charging points available
        if district_type == "Business district":
            parking_fee = round(random.uniform(1.1, 1.6) * random.uniform(7, 15), 2)
            available_charging_stations = random.randint(0, int(0.2 * 0.9 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(1.1, 1.2) * random.uniform(1.2, 1.5), 2)
                new_one_day_prices_list.append(new_price)
        elif district_type == "Office district":
            parking_fee = round(random.uniform(1.1, 1.6) * random.uniform(7, 15), 2)
            available_charging_stations = random.randint(0, int(0.2 * 0.1 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(1.1, 1.2) * random.uniform(1.2, 1.5), 2)
                new_one_day_prices_list.append(new_price)
        elif district_type == "Residential district":
            parking_fee = round(random.uniform(7, 15), 2)
            available_charging_stations = random.randint(0, int(0.2 * 0.8 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(1.1, 1.2) , 2)
                new_one_day_prices_list.append(new_price)

        parking_lots.append((destination.latitude, destination.longitude, distance, location_type, district_type, parking_fee, available_charging_stations, new_one_day_prices_list))

    for _ in range(num_downtown_parking_lots):
        location_type = "downtown"
        # Generate latitude and longitude within the city limits
        angle = random.uniform(0, 2 * 3.14159)
        distance = random.uniform(downtown_radius, suburb_radius)
        destination = geodesic(kilometers=distance).destination((center_lat, center_lon), angle)
        
        # Randomly select the attributes of business district, office district and Residential district
        district_type = random.choices(list(city_district_probs.keys()), weights=list(city_district_probs.values()))[0]
        
        new_one_day_prices_list = []
        # Set parking fees and the number of charging points available
        if district_type == "Business district":
            parking_fee = round(random.uniform(1.1, 1.6) * random.uniform(4, 8), 2)
            available_charging_stations = random.randint(0, int(0.5 * 0.9 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(1.2, 1.5), 2)
                new_one_day_prices_list.append(new_price)
        elif district_type == "Office district":
            parking_fee = round(random.uniform(1.1, 1.6) * random.uniform(4, 8), 2)
            available_charging_stations = random.randint(0, int(0.5 * 0.1 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(1.2, 1.5), 2)
                new_one_day_prices_list.append(new_price)
        elif district_type == "Residential district":
            parking_fee = round(random.uniform(4, 8), 2)
            available_charging_stations = random.randint(0, int(0.5 * 0.8 * 10))
            for price in one_day_prices_list:
                new_price = round(price, 2)
                new_one_day_prices_list.append(new_price)

        parking_lots.append((destination.latitude, destination.longitude, distance, location_type, district_type, parking_fee, available_charging_stations, new_one_day_prices_list))

    for _ in range(num_suburb_parking_lots):
        location_type = "suburb"
        # Generate latitude and longitude within the suburbs
        angle = random.uniform(0, 2 * 3.14159)
        distance = random.uniform(suburb_radius, city_radius)
        destination = geodesic(kilometers=distance).destination((center_lat, center_lon), angle)
        
        # Randomly select the attributes of business district, office district and Residential district
        district_type = random.choices(list(suburb_district_probs.keys()), weights=list(suburb_district_probs.values()))[0]
        
        new_one_day_prices_list = []
        # Set parking fees and the number of charging points available
        if district_type == "Business district":
            parking_fee = round(random.uniform(1.1, 1.6) * random.uniform(2, 5), 2)
            available_charging_stations = random.randint(0, int(0.8 * 0.9 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(0.8, 0.9) * random.uniform(1.2, 1.5), 2)
                new_one_day_prices_list.append(new_price)
            
        elif district_type == "Office district":
            parking_fee = round(random.uniform(1.1, 1.6) * random.uniform(2, 5), 2)
            available_charging_stations = random.randint(0, int(0.8 * 0.1 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(0.8, 0.9) * random.uniform(1.2, 1.5), 2)
                new_one_day_prices_list.append(new_price)
        elif district_type == "Residential district":
            parking_fee = round(random.uniform(2, 5), 2)
            available_charging_stations = random.randint(0, int(0.8 * 0.8 * 10))
            for price in one_day_prices_list:
                new_price = round(price * random.uniform(0.8, 0.9), 2)
                new_one_day_prices_list.append(new_price)

        parking_lots.append((destination.latitude, destination.longitude, distance, location_type, district_type, parking_fee, available_charging_stations, new_one_day_prices_list))

    return parking_lots

# Set the longitude, latitude and radius of the city center
# Suppose that the coordinates of the city center point are expressed in latitude and longitude as (30,120).
city_center_lat = 30
city_center_lon = 120
city_radius = 10  # 10 km radius of the city
downtown_radius = 2  # Downtown radius
suburb_radius = 6  # Suburban radius

# Generates longitude, latitude, properties, and parking fees for ten parking lots
num_parking_lots = 10
electricity_prices_file = "GR-data-11-20.csv"
parking_locations = generate_parking_locations(city_center_lat, city_center_lon, city_radius, downtown_radius, suburb_radius, num_parking_lots, electricity_prices_file)

# Print result
for i, location in enumerate(parking_locations):
    print(f"停车场 {i + 1}: 经度 - {location[1]}, 纬度 - {location[0]}, 距离城市中心的距离 - {location[2]} 公里, 距离属性 - {location[3]}, 区域属性 - {location[4]}, 停车费 - {location[5]} 元/小时, 可用充电桩 - {location[6]} 个, 一天电价 - {location[7]}")


停车场 1: 经度 - 120.0012187634839, 纬度 - 30.01523270883662, 距离城市中心的距离 - 1.6926740121980042 公里, 距离属性 - city_center, 区域属性 - Office district, 停车费 - 14.87 元/小时, 可用充电桩 - 0 个, 一天电价 - [90.98, 95.82, 96.05, 91.13, 82.17, 94.04, 81.72, 80.46, 79.41, 61.11, 59.62, 64.04, 57.83, 40.95, 63.14, 75.37, 97.37, 134.85, 89.01, 94.13, 80.03, 83.72, 75.57, 55.84]
停车场 2: 经度 - 120.00021685392004, 纬度 - 30.00302861894459, 距离城市中心的距离 - 0.3363812301901896 公里, 距离属性 - city_center, 区域属性 - Residential district, 停车费 - 11.09 元/小时, 可用充电桩 - 1 个, 一天电价 - [70.27, 70.67, 70.48, 66.91, 64.94, 66.3, 53.23, 63.67, 66.73, 48.26, 46.46, 45.16, 45.36, 32.4, 46.62, 60.51, 68.93, 105.25, 67.13, 75.99, 70.44, 67.98, 62.34, 47.75]
停车场 3: 经度 - 120.00021332887391, 纬度 - 30.014263015517127, 距离城市中心的距离 - 1.5812257780416776 公里, 距离属性 - city_center, 区域属性 - Office district, 停车费 - 8.82 元/小时, 可用充电桩 - 0 个, 一天电价 - [98.9, 85.9, 96.5, 92.54, 96.71, 93.6, 72.32, 91.86, 81.21, 59.52, 66.01, 60.41, 61.46, 38.73, 58.71, 91.17, 94.78, 150.65, 90.63, 101.86, 

In [ ]:
import math
from typing import Optional, Union

import numpy as np

import gym
from gym import logger, spaces
from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled

class StateSpace:
    def __init__(self, min_longitude, max_longitude, min_latitude, max_latitude, min_soc, max_soc, min_eprice, max_eprice):
        self.min_longitude = min_longitude
        self.max_longitude = max_longitude
        self.min_latitude = min_latitude
        self.max_latitude = max_latitude
        self.min_soc = min_soc
        self.max_soc = max_soc
        self.min_eprice = min_eprice
        self.max_eprice = max_eprice

    def normalize(self, longitude, latitude, soc, eprice):
        normalized_longitude = (longitude - self.min_longitude) / (self.max_longitude - self.min_longitude)
        normalized_latitude = (latitude - self.min_latitude) / (self.max_latitude - self.min_latitude)
        normalized_soc = (soc - self.min_soc) / (self.max_soc - self.min_soc)
        normalized_eprice = (eprice - self.min_eprice) / (self.max_eprice - self.min_eprice)
        return normalized_longitude, normalized_latitude, normalized_soc, normalized_eprice

    def denormalize(self, normalized_longitude, normalized_latitude, normalized_soc, normalized_eprice):
        longitude = normalized_longitude * (self.max_longitude - self.min_longitude) + self.min_longitude
        latitude = normalized_latitude * (self.max_latitude - self.min_latitude) + self.min_latitude
        soc = normalized_soc * (self.max_soc - self.min_soc) + self.min_soc
        eprice = normalized_eprice * (self.max_eprice - self.min_eprice) + self.min_eprice
        return longitude, latitude, soc, eprice


class CartPoleEnv(gym.Env[np.ndarray, Union[int, np.ndarray]]):
    """
    ### Action Space

    The action is a `ndarray` with shape `(1,)` which can take values `0~11` indicating the Driving destination.

    | Num | Action                 |
    |-----|------------------------|
    | 0   | Home                   |
    | 1   | Company                |
    | 2   | Parker 1               |
    | 3   | Parker 2               |
    | 4   | Parker 3               |
    | 5   | Parker 4               |
    | 6   | Parker 5               |
    | 7   | Parker 6               |
    | 8   | Parker 7               |
    | 9   | Parker 8               |
    | 10  | Parker 9               |
    | 11  | Parker 10              |

    ### Observation Space

    The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

    | Num | Observation           | Min                 | Max               |
    |-----|-----------------------|---------------------|-------------------|
    | 0   | Car Position          | (-180, -90)         | (180, 90)         |
    | 1   | SOC                   | 0                   | 77                |
    | 2   | Electricity Price     | 0                   | 200               |

    
    ### Rewards

    * r = EC * distance * eprice *

    ### Starting State

    SOC is 77, thr car position is the home of the car's owner

    ### Episode End

    The episode ends if any one of the following occurs:

    1. Termination: Reach the specified time
    2. Truncation: Episode length is greater than 500 (200 for v0)
    """

    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 50,
    }

    def __init__(self):
        # 生成上班时间，范围在七点到九点
        self.start_time = self.generate_random_time(7, 9)
        # 生成下班时间，范围在四点到六点
        self.end_time = self.generate_random_time(16, 18)

        # 定义经纬度范围和SOC范围
        min_longitude = -180.0
        max_longitude = 180.0
        min_latitude = -90.0
        max_latitude = 90.0
        min_soc = 0
        max_soc = 77
        min_eprice = 0
        max_eprice = 200
        self.current_step = 0
        self.df_prices = pd.read_csv("GR-data-11-20.csv", header=None, names=["DateTime", "ElectricityPrice"])

        #Tram energy consumption (per kilometer)
        EC = 0.145

        # 观察空间和动作空间的定义
        # 定义观察空间
        self.observation_space = spaces.Box(low=np.array([min_longitude, min_latitude, min_soc, min_eprice]),
                                            high=np.array([max_longitude, max_latitude, max_soc, max_eprice]),
                                            dtype=np.float32) 

        self.action_space = spaces.Discrete(12)

        self.state = None


    def step(self, action):
        longitude, latitude, soc, eprice = self.state
        
        start_time = self.start_time.hour
        end_time = self.end_time.hour



    def reset(
        self,
        *,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ):
        super().reset(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        low, high = utils.maybe_parse_reset_bounds(
            options, -0.05, 0.05  # default low
        )  # default high
        self.state = self.np_random.uniform(low=low, high=high, size=(4,))
        self.steps_beyond_terminated = None

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), {}



    def close(self):
        if self.screen is not None:
            import pygame

            pygame.display.quit()
            pygame.quit()
            self.isopen = False